# Preparación del entorno: instalaciones y librerías

In [1]:
#Instalaciones
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [136]:
#Librerías
import os
import numpy as np
import pandas as pd
import glob
import shutil
import torch
import torch.nn.functional as F
from torch import nn
from google.colab import drive
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, f1_score
from torch.optim import Adam
from sklearn.model_selection import ParameterGrid
from datasets import concatenate_datasets

# Dataset

In [76]:
#Carga del dataset de España
dataset_es = load_dataset("alttal/SA_opiniones_indumentaria_ES_Espanya_splits")

In [77]:
#Se eliminan columnas innecesarias
dataset_es = dataset_es.remove_columns(["id","__index_level_0__"])
dataset_es

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 2187
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 625
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 313
    })
})

# Métricas de evaluación

In [5]:
#Definimos las métricas para evaluar el modelo
#Este conjunto de métricas depende de la tarea.
#Para clasificación de textos se suelen utilizar accuracy, precision, recall y F1-score.

def compute_metrics(pred):
  y_true = pred.label_ids              # son las etiquetas reales
  y_pred = pred.predictions.argmax(-1) # son las predicciones
  acc = accuracy_score(y_true, y_pred)
  precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

  #métricas por clase
  precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(y_true, y_pred, average=None)

  #conversiones necesarias para evitar errores de registro en Tensorboard
  precision_per_class = list(precision_per_class)
  recall_per_class = list(recall_per_class)
  f1_per_class = list(f1_per_class)

  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall,
      'precision_per_class': precision_per_class,
      'recall_per_class': recall_per_class,
      'f1_per_class': f1_per_class
  }

# Documentación del progreso

In [6]:
#Montar unidad de disco
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#Ruta raíz para guardar cada experimento
path = '/content/drive/My Drive/TFM/TFM-Experimentos'

#Cambiar directorio actual de trabajo
os.chdir(path)

#Verificar directorio actual
print("Directorio actual:", os.getcwd())

Directorio actual: /content/drive/My Drive/TFM/TFM-Experimentos


In [8]:
#Definición de una función para eliminar logs antiguos (para evitar falta de almacenamiento)
def clean_old_logs(logging_dir, keep_last_n=1):
    log_files = sorted(glob.glob(os.path.join(logging_dir, '*')), key=os.path.getmtime)
    for log_file in log_files[:-keep_last_n]:
        os.remove(log_file)

In [9]:
#Ruta al directorio de logs
log_dir = path

#Eliminar archivos antiguos de TensorBoard (para evitar acumulación de demasiados logs)
for file in os.listdir(log_dir):
    if file.startswith('events.out.tfevents'):
        os.remove(os.path.join(log_dir, file))

# Modelo 1

##Carga del modelo 1 y adaptación de los datos

In [10]:
#Carga del modelo pre-entrenado
model1 = AutoModelForSequenceClassification.from_pretrained("finiteautomata/beto-sentiment-analysis", num_labels=3)

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [11]:
#A modo de información, vemos la configuración del modelo
model1.config

BertConfig {
  "_name_or_path": "finiteautomata/beto-sentiment-analysis",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "NEG",
    "1": "NEU",
    "2": "POS"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "NEG": 0,
    "NEU": 1,
    "POS": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.42.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31006
}

In [12]:
#¿Cómo son las etiquetas del modelo? Importante porque deben ser iguales a las del dataset
model1.config.id2label

{0: 'NEG', 1: 'NEU', 2: 'POS'}

In [78]:
#Las etiquetas de los datasets son strings. Se las cambia a valores numéricos
def label_to_int(label):
    if label == "POS":
        return 2
    elif label == "NEG":
        return 0
    else:
        return 1

dataset_es_1 = dataset_es.map(lambda x: {"label": label_to_int(x["label"])})

##Tokenización del dataset

In [79]:
#Carga del tokenizador específico
tokenizer1 = AutoTokenizer.from_pretrained("finiteautomata/beto-sentiment-analysis")

Primer test, sin especificar límites para la tokenización

In [68]:
#Se define tokenizador del primer modelo
def tokenize_function1(example):
    return tokenizer1(example["text"], padding=True, truncation=True)

In [69]:
#Tokenización del dataset
encoded_dataset_es_1 = dataset_es_1.map(tokenize_function1, batched=True)

Map:   0%|          | 0/2187 [00:00<?, ? examples/s]

Map:   0%|          | 0/625 [00:00<?, ? examples/s]

Map:   0%|          | 0/313 [00:00<?, ? examples/s]

In [73]:
#¿Cuál es la longitud de tokens para cada entrada?
token_lengths_train = [len(tokens) for tokens in encoded_dataset_es_1['train']['input_ids']]
token_lengths_eval = [len(tokens) for tokens in encoded_dataset_es_1['validation']['input_ids']]
token_lengths_test = [len(tokens) for tokens in encoded_dataset_es_1['test']['input_ids']]

#Calculamos la media para cada subconjunto de datos
media_tokens_train = sum(token_lengths_train) / len(token_lengths_train)
media_tokens_eval = sum(token_lengths_eval) / len(token_lengths_eval)
media_tokens_test = sum(token_lengths_test) / len(token_lengths_test)

print(f"La media de cantidad de tokens por entrada en el conjunto de datos tokenizado, subsplit train es: {media_tokens_train:.2f}")
print(f"La media de cantidad de tokens por entrada en el conjunto de datos tokenizado, subsplit validation es: {media_tokens_eval:.2f}")
print(f"La media de cantidad de tokens por entrada en el conjunto de datos tokenizado, subsplit test es: {media_tokens_test:.2f}")

#¿Cuál es la longitud máxima en cuanto a tokens de cada subconjunto de datos?
max_tokens_train = max(token_lengths_train)
max_tokens_eval = max(token_lengths_eval)
max_tokens_test = max(token_lengths_test)

#¿Cuál es la longitud máxima considerando todos los splits?
max_tokens_all = max(max_tokens_train, max_tokens_eval, max_tokens_test)
print(f"La longitud máxima, en cuanto a tokens, es: {max_tokens_all:.2f}")

La media de cantidad de tokens por entrada en el conjunto de datos tokenizado, subsplit train es: 284.84
La media de cantidad de tokens por entrada en el conjunto de datos tokenizado, subsplit validation es: 205.00
La media de cantidad de tokens por entrada en el conjunto de datos tokenizado, subsplit test es: 190.00
La longitud máxima, en cuanto a tokens, es: 314.00


Con esta información, se puede crear un tokenizador que se adapte mejor a los datos

In [80]:
#Se define tokenizador del primer modelo
def tokenize_function1(example):
    return tokenizer1(example["text"], max_length=314, padding='max_length', truncation=True)

In [90]:
#Tokenización del dataset
encoded_dataset_es_1 = dataset_es_1.map(tokenize_function1, batched=True)
#Luego de tokenizar se puede eliminar la columna "text" porque el modelo no la necesita
encoded_dataset_es_1 = encoded_dataset_es_1.remove_columns(["text"])
encoded_dataset_es_1

Map:   0%|          | 0/2187 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2187
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 625
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 313
    })
})

In [82]:
#Ejemplo de la tokenización en cada split
print(encoded_dataset_es_1['train'][0])
print(encoded_dataset_es_1['validation'][0])
print(encoded_dataset_es_1['test'][0])

{'label': 0, 'input_ids': [4, 1467, 1129, 15621, 1108, 3242, 1038, 1058, 1153, 17906, 1129, 7255, 17346, 1084, 7255, 26144, 30934, 1721, 1030, 22969, 1426, 1008, 1486, 1526, 6974, 21326, 9787, 1168, 1084, 1065, 2649, 1255, 5306, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [84]:
#¿Cuál es la longitud máxima considerando todos los splits?
max_tokens_all = max(max_tokens_train, max_tokens_eval, max_tokens_test)
print(f"La longitud máxima, en cuanto a tokens, es: {max_tokens_all:.2f}")

La longitud máxima, en cuanto a tokens, es: 314.00


## Modificación de pesos

Al tener un dataset desbalanceado en el cual la mayoría de las instancias pertenecen a la clase POS, y con una minoría de instancias etiquetadas como NEU, se ha observado en otros experimentos que el modelo tiene grandes dificultades a la hora de predecir la clase neutral porque, al recibir tan pocos ejemplos, no parece poder aprenderla.

Una alternativa en estos casos es modificar los pesos y penalizar al modelo para que aprenda mejor a identificar la clase minoritaria.

In [85]:
class_weights_dict = {}

#Iteración sobre cada split (train, test, validation)
for split in ['train', 'test', 'validation']:
    #Acceder a split
    dataset = encoded_dataset_es_1[split]

    #Convertir la columna 'label' a una serie de pandas
    labels_series = pd.Series(dataset['label'])

    #Calcular los pesos de clase
    class_weights = (1 - (labels_series.value_counts().sort_index() / len(labels_series))).values

    #Guardar los pesos en un diccionario
    class_weights_dict[split] = class_weights

print(class_weights_dict)

{'train': array([0.60265203, 0.85550983, 0.54183813]), 'test': array([0.6032, 0.856 , 0.5408]), 'validation': array([0.60063898, 0.85623003, 0.54313099])}


In [86]:
#El trainer se basa en pytorch, por eso se modifica de array numpy a tensores pytorch
class_weights = torch.from_numpy(class_weights).float().to("cuda")
class_weights

tensor([0.6006, 0.8562, 0.5431], device='cuda:0')

In [91]:
#Siguiendo la documentación, el trainer puede calcular la pérdida si se le provee un argumento 'labels'
#Fuente: https://huggingface.co/docs/transformers/main_classes/trainer & https://www.youtube.com/watch?v=u--UVvH-LIQ
#El que tenemos se llama 'label', por eso se lo modifica
encoded_dataset_es_1 = encoded_dataset_es_1.rename_column("label", "labels")

In [92]:
#Creamos una clase que es subclase de la clase Trainer
#Desde ahí definimos cómo calcular la pérdida (loss function)
class WeightedLossTrainer(Trainer):
  def compute_metrics(self, model, inputs, return_outputs=False):
    #Introducir inputs al modelo
    outputs = model(**inputs)
    logits = outputs.get("logits")
    #Extraer labels
    labels = inputs.get("labels")
    #Definir función de pérdida con los nuevos pesos
    loss = loss_func = nn.CrossEntropyLoss(weight=class_weights)
    #Calcular pérdida
    loss = loss_func(logits, labels)
    return (loss, outputs) if return_outputs else loss

## Experimento 1.1

In [22]:
#Documentación
#Definición de directorios para el primer experimento
output_dir = "./outputs/weights_model1_lit"
shutil.make_archive("./outputs/weights_model1_lit", 'zip', output_dir)
logging_dir = "./logs/weights_model1_lit"
shutil.make_archive("./logs/weights_model1_lit", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros recomendados

In [23]:
#Definición de EarlyStopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,    # Número de épocas para esperar sin mejora
    early_stopping_threshold=0.0  # Umbral para la mejora mínima
)

In [24]:
#Definición de hiperparámetros
training_args_weights_model1_lit = TrainingArguments(
    output_dir=output_dir,           # Directorio de salida
    logging_dir=logging_dir,         # Directorio para los logs
    eval_strategy="steps",           # Estrategia de evaluación
    save_strategy="steps",           # Estrategia de guardado (debe coincidir con la de evaluación)
    save_steps=1000,                 # Número de pasos entre cada guardado
    logging_strategy="steps",        # Estrategia de registro
    logging_steps=1000,              # Número de pasos entre cada registro
    num_train_epochs=3,              # Número de épocas de entrenamiento
    per_device_train_batch_size=16,  # Tamaño del batch de entrenamiento (podría ser también 32)
    per_device_eval_batch_size=16,   # Tamaño del batch de evaluación (podría ser también 32)
    learning_rate=3e-5,              # Tasa de aprendizaje
    weight_decay=0.01,               # Decadencia de peso (podría ser también 0.1)
    warmup_ratio=0.1,                # Proporción de épocas de calentamiento
    save_total_limit=1,              # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
    seed=42,                         # Semilla para garantizar reproducibilidad
    fp16=True,                       # Uso de fp16 para acelerar el entrenamiento
    load_best_model_at_end=True,     # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",      # Métrica objetivo, a optimizar para encontrar el mejor modelo
)

In [25]:
#Definición de optimizador
optimizer = Adam(model1.parameters(), lr=3e-5, weight_decay=0.01)

In [26]:
#Definición de un objeto de la clase Trainer para el primer experimento con el modelo 1
trainer_weights_model1_lit = WeightedLossTrainer(
    model = model1,
    args = training_args_weights_model1_lit,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer1,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer1),
    optimizers = (optimizer, None),
    callbacks = [early_stopping])

Entrenamiento

In [27]:
trainer_weights_model1_lit.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=411, training_loss=0.568206787109375, metrics={'train_runtime': 35.007, 'train_samples_per_second': 187.419, 'train_steps_per_second': 11.74, 'total_flos': 1058699531222676.0, 'train_loss': 0.568206787109375, 'epoch': 3.0})

Validación

In [28]:
trainer_weights_model1_lit.evaluate()

Trainer is attempting to log a value of "[0.8951612903225806, 0.5365853658536586, 0.8851351351351351]" of type <class 'list'> for key "eval/precision_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.888, 0.4888888888888889, 0.916083916083916]" of type <class 'list'> for key "eval/recall_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.891566265060241, 0.5116279069767442, 0.9003436426116838]" of type <class 'list'> for key "eval/f1_per_class" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.41863298416137695,
 'eval_accuracy': 0.8434504792332268,
 'eval_f1': 0.8409526512458606,
 'eval_precision': 0.8390282015273532,
 'eval_recall': 0.8434504792332268,
 'eval_precision_per_class': [0.8951612903225806,
  0.5365853658536586,
  0.8851351351351351],
 'eval_recall_per_class': [0.888, 0.4888888888888889, 0.916083916083916],
 'eval_f1_per_class': [0.891566265060241,
  0.5116279069767442,
  0.9003436426116838],
 'eval_runtime': 0.5449,
 'eval_samples_per_second': 574.463,
 'eval_steps_per_second': 36.707,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [29]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_weights_model1_lit.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.90      0.89      0.89       125
           1       0.54      0.49      0.51        45
           2       0.89      0.92      0.90       143

    accuracy                           0.84       313
   macro avg       0.77      0.76      0.77       313
weighted avg       0.84      0.84      0.84       313


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.90   0.89     0.89
    1      0.54   0.49     0.51
    2      0.89   0.92     0.90


In [30]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [31]:
#Obtención de predicciones del conjunto de test
predictions = trainer_weights_model1_lit.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.88      0.87      0.88       248
           1       0.51      0.44      0.47        90
           2       0.87      0.92      0.89       287

    accuracy                           0.83       625
   macro avg       0.75      0.74      0.75       625
weighted avg       0.82      0.83      0.83       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.88   0.87     0.88
    1      0.51   0.44     0.47
    2      0.87   0.92     0.89


In [32]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Grid Search

Búsqueda de hiperparámetros optimizados con la técnica Grid Search:

In [33]:
#Definición del espacio de búsqueda
param_grid = {
    'num_train_epochs': [3, 10],
    'per_device_train_batch_size': [16, 32],
    'per_device_eval_batch_size': [16, 32],
    'learning_rate': [1e-5, 2e-5, 3e-5, 5e-5, 1e-4],
    'weight_decay': [0.01, 0.1],
    'warmup_ratio': [0.06, 0.1],
}

grid = ParameterGrid(param_grid)

In [34]:
#Función para calcular el f1-score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {
        'f1': f1_score(labels, predictions, average='weighted')
    }

In [35]:
#Inicialización de variables para guardar los mejores resultados
best_score = 0
best_params = None

In [36]:
#Documentación
#Definición de directorios para el primer experimento
output_dir = "./outputs/weights_model1_search"
shutil.make_archive("./outputs/weights_model1_search", 'zip', output_dir)
logging_dir = "./logs/weights_model1_search"
shutil.make_archive("./logs/weights_model1_search", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

In [39]:
#Bucle para probar los diferentes parámetros
for params in grid:
    print(f"Entrenando con parámetros: {params}")

    #Definición de hiperparámetros
    training_args_weights_model1_search = TrainingArguments(
        output_dir=output_dir,                                                  # Directorio de salida
        logging_dir=logging_dir,                                                # Directorio para los logs
        eval_strategy="steps",                                                  # Estrategia de evaluación
        save_strategy="steps",                                                  # Estrategia de guardado (debe coincidir con la de evaluación)
        save_steps=1000,                                                        # Número de pasos entre cada guardado
        logging_strategy="steps",                                               # Estrategia de registro
        logging_steps=1000,                                                     # Número de pasos entre cada registro
        num_train_epochs=params['num_train_epochs'],                            # Número de épocas de entrenamiento
        per_device_train_batch_size=params['per_device_train_batch_size'],      # Tamaño del batch de entrenamiento
        per_device_eval_batch_size=params['per_device_eval_batch_size'],        # Tamaño del batch de evaluación
        learning_rate=params['learning_rate'],                                  # Tasa de aprendizaje
        weight_decay=params['weight_decay'],                                    # Decadencia de peso
        warmup_ratio=params['warmup_ratio'],                                    # Proporción de épocas de calentamiento
        save_total_limit=1,                                                     # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
        seed=42,                                                                # Semilla para garantizar reproducibilidad
        fp16=True,                                                              # Uso de fp16 para acelerar el entrenamiento
        load_best_model_at_end=True,                                            # Cargar el mejor modelo al final del entrenamiento
        metric_for_best_model='f1',                                             # Métrica objetivo, a optimizar para encontrar el mejor modelo
    )

    #Definición de un objeto de la clase Trainer para la búsqueda con el modelo 1
    trainer_weights_model1_search = WeightedLossTrainer(
        model=model1,
        args=training_args_weights_model1_search,
        train_dataset=encoded_dataset_es_1['train'],
        eval_dataset=encoded_dataset_es_1['validation'],
        tokenizer=tokenizer1,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer1),
        compute_metrics=compute_metrics,
        optimizers = (optimizer, None),
        callbacks = [early_stopping]
    )

    #Entrenamiento
    trainer_weights_model1_search.train()

    #Limpiar logs antiguos después del entrenamiento en cada iteración
    clean_old_logs(logging_dir, keep_last_n=1)

    #Validación
    eval_result = trainer_weights_model1_search.evaluate()

    #Guardamos métrica objetivo
    score = eval_result.get('eval_f1', 0)

    #Siempre que la métrica objetivo sea mejor que el score guardado hasta el momento, se lo actualiza
    if score > best_score:
        best_score = score
        best_params = params

#Vemos los resultados del Grid Search
print(f"Mejores hiperparámetros: {best_params}")
print(f"Mejor puntuación F1: {best_score}")

Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'warmup_ratio': 0.06, 'weight_decay': 0.01}


Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'warmup_ratio': 0.06, 'weight_decay': 0.1}


Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'warmup_ratio': 0.1, 'weight_decay': 0.01}


...

Entrenando con parámetros: {'learning_rate': 0.0001, 'num_train_epochs': 10, 'per_device_eval_batch_size': 32, 'per_device_train_batch_size': 32, 'warmup_ratio': 0.1, 'weight_decay': 0.01}


Step,Training Loss,Validation Loss


Entrenando con parámetros: {'learning_rate': 0.0001, 'num_train_epochs': 10, 'per_device_eval_batch_size': 32, 'per_device_train_batch_size': 32, 'warmup_ratio': 0.1, 'weight_decay': 0.1}


Step,Training Loss,Validation Loss


Mejores hiperparámetros: {'learning_rate': 1e-05, 'num_train_epochs': 3, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'warmup_ratio': 0.06, 'weight_decay': 0.01}
Mejor puntuación F1: 0.8086963150355885


## Experimento 1.2.

In [40]:
#Documentación
#Definición de directorios para el segundo experimento
output_dir = "./outputs/weights_model1_opt"
shutil.make_archive("./outputs/weights_model1_opt", 'zip', output_dir)
logging_dir = "./logs/weights_model1_opt"
shutil.make_archive("./logs/weights_model1_opt", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros optimizados

In [41]:
#Definición de EarlyStopping
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,    # Número de épocas para esperar sin mejora
    early_stopping_threshold=0.0  # Umbral para la mejora mínima
)

In [42]:
#Definición de hiperparámetros (basados en Grid Search)
training_args_weights_model1_opt = TrainingArguments(
    output_dir=output_dir,                                # Directorio de salida
    logging_dir=logging_dir,                              # Directorio para los logs
    eval_strategy="steps",                                # Estrategia de evaluación
    save_strategy="steps",                                # Estrategia de guardado (debe coincidir con la de evaluación)
    save_steps=1000,                                      # Número de pasos entre cada guardado
    logging_strategy="steps",                             # Estrategia de registro
    logging_steps=1000,                                   # Número de pasos entre cada registro
    num_train_epochs=3,                                   # Número de épocas de entrenamiento
    per_device_train_batch_size=16,                       # Tamaño del batch de entrenamiento
    per_device_eval_batch_size=16,                        # Tamaño del batch de evaluación
    learning_rate=1e-05,                                  # Tasa de aprendizaje
    weight_decay=0.01,                                    # Decadencia de peso
    warmup_ratio=0.06,                                    # Proporción de épocas de calentamiento
    save_total_limit=1,                                   # Número máximo de checkpoints a guardar (para evitar problemas de almacenamiento)
    seed=42,                                              # Semilla para garantizar reproducibilidad
    fp16=True,                                            # Uso de fp16 para acelerar el entrenamiento
    load_best_model_at_end=True,                          # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",                           # Métrica objetivo, a optimizar para encontrar el mejor modelo
)

In [43]:
#Definición de optimizador
optimizer = Adam(model1.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])

In [44]:
#Definición de un objeto de la clase Trainer para el segundo experimento con el modelo 1
trainer_weights_model1_opt = WeightedLossTrainer(
    model = model1,
    args = training_args_weights_model1_opt,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer1,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer1),
    optimizers = (optimizer, None),
    callbacks = [early_stopping])

Entrenamiento

In [45]:
trainer_weights_model1_opt.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=411, training_loss=0.985184421214454, metrics={'train_runtime': 33.0211, 'train_samples_per_second': 198.691, 'train_steps_per_second': 12.447, 'total_flos': 1058699531222676.0, 'train_loss': 0.985184421214454, 'epoch': 3.0})

Validación

In [46]:
trainer_weights_model1_opt.evaluate()

{'eval_loss': 1.0717133283615112,
 'eval_f1': 0.6323537021186424,
 'eval_runtime': 0.6467,
 'eval_samples_per_second': 483.982,
 'eval_steps_per_second': 30.925,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [47]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_weights_model1_opt.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.70      0.72      0.71       125
           1       0.00      0.00      0.00        45
           2       0.68      0.87      0.76       143

    accuracy                           0.69       313
   macro avg       0.46      0.53      0.49       313
weighted avg       0.59      0.69      0.63       313


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.70   0.72     0.71
    1      0.00   0.00     0.00
    2      0.68   0.87     0.76


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [48]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [49]:
#Obtención de predicciones del conjunto de test
predictions = trainer_weights_model1_opt.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.69      0.67      0.68       248
           1       0.00      0.00      0.00        90
           2       0.64      0.86      0.73       287

    accuracy                           0.66       625
   macro avg       0.44      0.51      0.47       625
weighted avg       0.57      0.66      0.61       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.69   0.67     0.68
    1      0.00   0.00     0.00
    2      0.64   0.86     0.73


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [50]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

## Experimento extra 1.3

In [52]:
#Documentación
#Definición de directorios para el tercer experimento
output_dir = "./outputs/weights_model1_est"
shutil.make_archive("./outputs/weights_model1_est", 'zip', output_dir)
logging_dir = "./logs/weights_model1_est"
shutil.make_archive("./logs/weights_model1_est", 'zip', output_dir)

#Si directorios no existen, crearlos
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

Entrenamiento con hiperparámetros originales

In [53]:
#Definición de hiperparámetros estándar
training_args_weights_model1_est = TrainingArguments(
    output_dir=output_dir,
    logging_dir=logging_dir,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    logging_strategy="steps",
    logging_steps=1000,
    seed=42,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [93]:
#Definición de un objeto de la clase Trainer para el tercer experimento con el modelo 1
trainer_weights_model1_est = WeightedLossTrainer(
    model = model1,
    args = training_args_weights_model1_est,
    train_dataset = encoded_dataset_es_1['train'],
    eval_dataset = encoded_dataset_es_1['validation'],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer1,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer1),
    optimizers = (optimizer, None),
    callbacks = [early_stopping])

Entrenamiento

In [94]:
trainer_weights_model1_est.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=822, training_loss=1.0110978546513838, metrics={'train_runtime': 62.3249, 'train_samples_per_second': 105.271, 'train_steps_per_second': 13.189, 'total_flos': 1058699531222676.0, 'train_loss': 1.0110978546513838, 'epoch': 3.0})

Validación

In [56]:
trainer_weights_model1_est.evaluate()

{'eval_loss': 1.081263780593872,
 'eval_f1': 0.2865450367131887,
 'eval_runtime': 0.6437,
 'eval_samples_per_second': 486.219,
 'eval_steps_per_second': 31.068,
 'epoch': 3.0}

Reporte de clasificación (sobre conjunto de validación)

In [95]:
#Obtención de predicciones del conjunto de validación
predictions = trainer_weights_model1_est.predict(encoded_dataset_es_1['validation'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       125
           1       0.00      0.00      0.00        45
           2       0.46      1.00      0.63       143

    accuracy                           0.46       313
   macro avg       0.15      0.33      0.21       313
weighted avg       0.21      0.46      0.29       313


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.00   0.00     0.00
    1      0.00   0.00     0.00
    2      0.46   1.00     0.63


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  >>> fbeta_score(y_true, y_pred, average='macro', beta=0.5)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  >>> fbeta_score(y_true, y_pred, average='macro', beta=0.5)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  >>> fbeta_score(y_true, y_pred, average='macro', beta=0.5)
/usr/local/lib/python3.10/dist-packages/sklear

In [96]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

Test

Reporte de clasificación (sobre conjunto de test)

In [97]:
#Obtención de predicciones del conjunto de test
predictions = trainer_weights_model1_est.predict(encoded_dataset_es_1['test'])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       248
           1       0.00      0.00      0.00        90
           2       0.46      1.00      0.63       287

    accuracy                           0.46       625
   macro avg       0.15      0.33      0.21       625
weighted avg       0.21      0.46      0.29       625


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.00   0.00     0.00
    1      0.00   0.00     0.00
    2      0.46   1.00     0.63


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  >>> fbeta_score(y_true, y_pred, average='macro', beta=0.5)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  >>> fbeta_score(y_true, y_pred, average='macro', beta=0.5)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  >>> fbeta_score(y_true, y_pred, average='macro', beta=0.5)
/usr/local/lib/python3.10/dist-packages/sklear

In [98]:
#Limpiar logs antiguos
clean_old_logs(logging_dir, keep_last_n=1)

# Mejor modelo, nuevo dataset

## Dataset

In [114]:
#Carga del dataset de España
dataset_ar = load_dataset("alttal/SA_opiniones_indumentaria_ES_Argentina_splits")
dataset_ar

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'text', '__index_level_0__'],
        num_rows: 2187
    })
    test: Dataset({
        features: ['id', 'label', 'text', '__index_level_0__'],
        num_rows: 625
    })
    validation: Dataset({
        features: ['id', 'label', 'text', '__index_level_0__'],
        num_rows: 313
    })
})

In [116]:
#Unimos los splits: sólo necesitaremos uno para el test
dataset_ar = concatenate_datasets([dataset_ar['train'], dataset_ar['test'], dataset_ar['validation']])

#Ejemplo
print(dataset_ar)

Dataset({
    features: ['id', 'label', 'text', '__index_level_0__'],
    num_rows: 3125
})


In [117]:
#Se eliminan columnas innecesarias
dataset_ar = dataset_ar.remove_columns(["id","__index_level_0__"])
dataset_ar

Dataset({
    features: ['label', 'text'],
    num_rows: 3125
})

Adaptación de los datos

In [118]:
#Las etiquetas de los datasets son strings. Se las cambia a valores numéricos
def label_to_int(label):
    if label == "POS":
        return 2
    elif label == "NEG":
        return 0
    else:
        return 1

dataset_ar_1 = dataset_ar.map(lambda x: {"label": label_to_int(x["label"])})

Map:   0%|          | 0/3125 [00:00<?, ? examples/s]

## Tokenización

In [119]:
#Tokenización del dataset
encoded_dataset_ar_1 = dataset_ar_1.map(tokenize_function1, batched=True)
#Luego de tokenizar se puede eliminar la columna "text" porque el modelo no la necesita
encoded_dataset_ar_1 = encoded_dataset_ar_1.remove_columns(["text"])
encoded_dataset_ar_1

Map:   0%|          | 0/3125 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3125
})

## Experimento con dataset de Argentina

In [137]:
#Obtención de predicciones del conjunto de test
predictions = trainer_weights_model1_lit.predict(encoded_dataset_ar_1)
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

print("Predicciones:")

#Generación del reporte (diccionario)
reporte_dict = classification_report(y_true, y_pred, output_dict=True)

#Reporte completo (texto)
print("Reporte de Clasificación:")
print(classification_report(y_true, y_pred))

#Preparación de datos para el DataFrame
metrics_per_class = []
for label, metrics in reporte_dict.items():
    if label.isdigit():
        metrics_class = {
            "Clase": label,
            "Precisión": f"{metrics['precision']:.2f}",
            "Recall": f"{metrics['recall']:.2f}",
            "F1-score": f"{metrics['f1-score']:.2f}"
        }
        metrics_per_class.append(metrics_class)

#Creación de DataFrame con métricas por clase
df_metrics_per_class = pd.DataFrame(metrics_per_class)

#Tabla de métricas por clase
print("\nMétricas por Clase:")
print(df_metrics_per_class.to_string(index=False))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  >>> fbeta_score(y_true, y_pred, average='macro', beta=0.5)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  >>> fbeta_score(y_true, y_pred, average='macro', beta=0.5)


Predicciones:
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       703
           1       0.00      0.00      0.00       333
           2       0.67      1.00      0.80      2089

    accuracy                           0.67      3125
   macro avg       0.22      0.33      0.27      3125
weighted avg       0.45      0.67      0.54      3125


Métricas por Clase:
Clase Precisión Recall F1-score
    0      0.00   0.00     0.00
    1      0.00   0.00     0.00
    2      0.67   1.00     0.80


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  >>> fbeta_score(y_true, y_pred, average='macro', beta=0.5)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  >>> fbeta_score(y_true, y_pred, average='macro', beta=0.5)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  >>> fbeta_score(y_true, y_pred, average='macro', beta=0.5)
/usr/local/lib/python3.10/dist-packages/sklear